PROCESS DATA
1) Train LightGBM  
1.1) gridsearch  
1.2) use the best params then do k fold ? then out of fold predictions save them  
2) Train XGBoost   
2.1) gridsearch   
2.2) use the best params then do k fold? then out of fold predictions save them 
3) Train CatBoost 
3.1) gridsearch   
3.2) use the best params then do k fold? then out of fold predictions save them 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score 
from catboost import CatBoostClassifier

Process Data

In [2]:
# pd optiosn to read all columns 
pd.set_option('display.max_columns', None) 

In [3]:
df = pd.read_parquet('CLEANED_V9.parquet') 
df

,Airline Code,Aircraft Registration,Operator,Type Code,Mode S,Serial Number,Age(years),FROM,TO,Arrival_Delayed,STD_temp_scaled,STD_dwpt_scaled,STD_rhum_scaled,STD_prcp_scaled,STD_snow_scaled,STD_wdir_scaled,STD_wspd_scaled,STD_wpgt_scaled,STD_pres_scaled,STD_tsun_scaled,STD_coco_scaled,STA_temp_scaled,STA_dwpt_scaled,STA_rhum_scaled,STA_prcp_scaled,STA_snow_scaled,STA_wdir_scaled,STA_wspd_scaled,STA_wpgt_scaled,STA_pres_scaled,STA_tsun_scaled,STA_coco_scaled,STD_UTC_time_of_day_cosine,STD_UTC_time_of_year_cosine,STD_UTC_day_of_year,STD_UTC_week_of_year,STD_UTC_weekday,STD_UTC_hour_of_day,STA_UTC_time_of_day_cosine,STA_UTC_time_of_year_cosine,STA_UTC_day_of_year,STA_UTC_week_of_year,STA_UTC_weekday,STA_UTC_hour_of_day,final_pred_dep,Y_arrival
0,3u-csc,b-30cr,Sichuan Airlines,A20N,781848,8873.0,5.0,YIN,TFU,True,0.605435,0.672010,0.62,NaN,NaN,0.125000,0.0360,NaN,0.677922,NaN,NaN,0.776596,0.861596,0.66,NaN,NaN,1.000000,0.045570,NaN,0.590314,NaN,NaN,0.555570,-0.492533,244,35,5,3,0.021815,-0.492533,244.0,35.0,5.0,5.0,1,True
1,3u-csc,b-30cr,Sichuan Airlines,A20N,781848,8873.0,5.0,TFU,YIN,False,0.739130,0.863132,0.79,NaN,NaN,0.055556,0.0432,NaN,NaN,NaN,NaN,0.613830,0.668329,0.62,NaN,NaN,0.125000,0.037975,NaN,0.683246,NaN,NaN,0.625923,-0.507430,243,35,4,20,0.707107,-0.492533,244.0,35.0,5.0,3.0,1,False
2,3u-csc,b-30cr,Sichuan Airlines,A20N,781848,8873.0,5.0,URC,TFU,True,0.652174,0.727497,0.64,NaN,NaN,0.530556,0.0144,NaN,0.688312,NaN,NaN,0.755319,0.860349,0.74,NaN,NaN,1.000000,0.045570,NaN,NaN,NaN,NaN,-0.087156,-0.507430,243,35,4,17,0.362438,-0.507430,243.0,35.0,4.0,19.0,1,True
3,3u-csc,b-30cr,Sichuan Airlines,A20N,781848,8873.0,5.0,TFU,URC,False,0.782609,0.852035,0.59,NaN,NaN,0.083333,0.0576,NaN,NaN,NaN,NaN,0.712766,0.724439,0.47,NaN,NaN,0.944444,0.045570,NaN,NaN,NaN,NaN,0.675590,-0.507430,243,35,4,3,-0.642788,-0.507430,243.0,35.0,4.0,8.0,1,False
4,3u-csc,b-30cr,Sichuan Airlines,A20N,781848,8873.0,5.0,ZHA,TFU,False,0.744565,0.898890,0.91,NaN,NaN,0.166667,0.0288,NaN,0.612338,NaN,0.28,0.787234,0.862843,0.63,NaN,NaN,0.111111,0.060759,NaN,NaN,NaN,NaN,0.999762,-0.522178,242,35,3,23,0.854912,-0.507430,243.0,35.0,4.0,2.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11669782,SQ-SIA,9V-SDA,Singapore Airlines,B78X,76CC81,67157.0,0.0,NGO,SIN,False,0.717391,0.789149,0.61,0.0,NaN,0.777778,0.0520,NaN,0.655844,NaN,0.08,0.787234,0.887781,0.71,0.0,NaN,0.333333,0.047257,NaN,0.646597,NaN,0.08,0.793353,-0.084835,269,39,2,2,0.793353,-0.084835,269.0,39.0,2.0,2.0,1,False
11669783,SQ-SIA,9V-SDA,Singapore Airlines,B78X,76CC81,67157.0,0.0,CHS,NGO,True,0.710870,0.868064,0.94,0.0,NaN,0.416667,0.0376,NaN,0.655195,NaN,0.04,0.680851,0.774314,0.73,0.0,NaN,0.138889,0.054852,NaN,0.667539,NaN,0.08,0.258819,-0.101962,268,39,1,5,0.237686,-0.101962,268.0,39.0,1.0,18.0,1,True
11669784,SQ-SIA,9V-SDA,Singapore Airlines,B78X,76CC81,67157.0,0.0,CHS,CHS,True,0.627174,0.755857,0.86,0.0,NaN,0.027778,0.0448,NaN,0.685714,NaN,0.04,0.670213,0.761845,0.73,0.0,NaN,0.083333,0.054852,NaN,0.695026,NaN,0.04,-0.923880,-0.336289,254,37,1,10,-0.928810,-0.336289,254.0,37.0,1.0,13.0,1,True
11669785,SQ-SIA,9V-SDA,Singapore Airlines,B78X,76CC81,67157.0,0.0,CHS,CHS,True,0.790217,0.833539,0.52,0.0,NaN,0.472222,0.0520,NaN,0.673377,NaN,0.28,0.794681,0.831671,0.52,0.0,NaN,0.472222,0.054852,NaN,0.678665,NaN,0.28,-0.555570,-0.477489,245,35,6,15,-0.492424,-0.477489,245.0,35.0,6.0,16.0,1,True


In [4]:
y = df['Arrival_Delayed']  # target variable 

# drop columns that are not needed
columns_to_drop = [col for col in df.columns if col.startswith('ATA') or col.startswith('ATD') or col == 'Arrival_Delayed']
df = df.drop(columns=columns_to_drop)
#duplicated column of arrival_delayed 
X = df.drop(columns=['Y_arrival']) 




In [5]:
X.columns

Index(['Airline Code', 'Aircraft Registration', 'Operator', 'Type Code',
       'Mode S', 'Serial Number', 'Age(years)', 'FROM', 'TO',
       'STD_temp_scaled', 'STD_dwpt_scaled', 'STD_rhum_scaled',
       'STD_prcp_scaled', 'STD_snow_scaled', 'STD_wdir_scaled',
       'STD_wspd_scaled', 'STD_wpgt_scaled', 'STD_pres_scaled',
       'STD_tsun_scaled', 'STD_coco_scaled', 'STA_temp_scaled',
       'STA_dwpt_scaled', 'STA_rhum_scaled', 'STA_prcp_scaled',
       'STA_snow_scaled', 'STA_wdir_scaled', 'STA_wspd_scaled',
       'STA_wpgt_scaled', 'STA_pres_scaled', 'STA_tsun_scaled',
       'STA_coco_scaled', 'STD_UTC_time_of_day_cosine',
       'STD_UTC_time_of_year_cosine', 'STD_UTC_day_of_year',
       'STD_UTC_week_of_year', 'STD_UTC_weekday', 'STD_UTC_hour_of_day',
       'STA_UTC_time_of_day_cosine', 'STA_UTC_time_of_year_cosine',
       'STA_UTC_day_of_year', 'STA_UTC_week_of_year', 'STA_UTC_weekday',
       'STA_UTC_hour_of_day', 'final_pred_dep'],
      dtype='object')

In [6]:
print (X.dtypes)

# Convert all float64 columns to float32
float_cols = X.select_dtypes(include=['float64']).columns
X[float_cols] = X[float_cols].astype(np.float32)

# Convert all int64 columns to int32 (if range is small enough)
int_cols = X.select_dtypes(include=['int64']).columns
X[int_cols] = X[int_cols].astype(np.int32)

# Convert categorical variables to category dtype
cat_cols = X.select_dtypes(include=['object', 'category']).columns
X[cat_cols] = X[cat_cols].astype('category')


# Handle missing values in categorical columns by adding "Missing" as a valid category
for col in cat_cols:
    X[col] = X[col].cat.add_categories("Missing")
    X[col] = X[col].fillna("Missing")


Airline Code                   category
Aircraft Registration          category
Operator                       category
Type Code                      category
Mode S                         category
Serial Number                  category
Age(years)                      float32
FROM                           category
TO                             category
STD_temp_scaled                 float64
STD_dwpt_scaled                 float64
STD_rhum_scaled                 float64
STD_prcp_scaled                 float64
STD_snow_scaled                 float64
STD_wdir_scaled                 float64
STD_wspd_scaled                 float64
STD_wpgt_scaled                 float64
STD_pres_scaled                 float64
STD_tsun_scaled                 float64
STD_coco_scaled                 float64
STA_temp_scaled                 float64
STA_dwpt_scaled                 float64
STA_rhum_scaled                 float64
STA_prcp_scaled                 float64
STA_snow_scaled                 float64


In [7]:
X.dtypes

Airline Code                   category
Aircraft Registration          category
Operator                       category
Type Code                      category
Mode S                         category
Serial Number                  category
Age(years)                      float32
FROM                           category
TO                             category
STD_temp_scaled                 float32
STD_dwpt_scaled                 float32
STD_rhum_scaled                 float32
STD_prcp_scaled                 float32
STD_snow_scaled                 float32
STD_wdir_scaled                 float32
STD_wspd_scaled                 float32
STD_wpgt_scaled                 float32
STD_pres_scaled                 float32
STD_tsun_scaled                 float32
STD_coco_scaled                 float32
STA_temp_scaled                 float32
STA_dwpt_scaled                 float32
STA_rhum_scaled                 float32
STA_prcp_scaled                 float32
STA_snow_scaled                 float32


In [8]:
y.dtypes

dtype('bool')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

CHECKING FOR UNIQUE VALUES BEFORE I RUN 

In [10]:
# Check categorical columns
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    print(f"{col}: {X_train[col].nunique()} unique values")

# Check numerical columns
num_cols = X_train.select_dtypes(include=['float64', 'float32']).columns
for col in num_cols:
    print(f"{col}: {X_train[col].nunique()} unique values")


Airline Code: 78 unique values
Aircraft Registration: 9745 unique values
Operator: 113 unique values
Type Code: 64 unique values
Mode S: 9560 unique values
Serial Number: 9404 unique values
FROM: 1876 unique values
TO: 1847 unique values
Age(years): 36 unique values
STD_temp_scaled: 855 unique values
STD_dwpt_scaled: 731 unique values
STD_rhum_scaled: 100 unique values
STD_prcp_scaled: 323 unique values
STD_snow_scaled: 53 unique values
STD_wdir_scaled: 361 unique values
STD_wspd_scaled: 289 unique values
STD_wpgt_scaled: 352 unique values
STD_pres_scaled: 909 unique values
STD_tsun_scaled: 61 unique values
STD_coco_scaled: 25 unique values
STA_temp_scaled: 857 unique values
STA_dwpt_scaled: 725 unique values
STA_rhum_scaled: 101 unique values
STA_prcp_scaled: 320 unique values
STA_snow_scaled: 60 unique values
STA_wdir_scaled: 361 unique values
STA_wspd_scaled: 301 unique values
STA_wpgt_scaled: 353 unique values
STA_pres_scaled: 913 unique values
STA_tsun_scaled: 61 unique values
STA

GRID SEARCH FOR LGB (SKIP THIS)

In [ ]:
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'device': 'cpu',
    'verbosity': 1,
    'max_bin': 255,       # A reasonable bin size for CPU
    'n_jobs': 16          # Use all 16 threads in LightGBM
}



lgb_grid = {
    'num_leaves': [31, ],
    'learning_rate': [0.01, 0.05],
    'n_estimators': [10, 20]
}
# If your dataset is large, you might reduce cv=3 or cv=5 to manage runtime.
# For parallel hyperparameter searches, you can set n_jobs=4 or 8 in GridSearchCV.
# Using all 16 can be very heavy if each fold also tries to use multiple threads.
lgb_gs = GridSearchCV(
    estimator=lgb.LGBMClassifier(**lgb_params),
    param_grid=lgb_grid,
    cv=3,
    scoring='accuracy',
    verbose=3,
    n_jobs=4  # parallel jobs for hyperparameter search
)

lgb_gs.fit(X_train, y_train)
print("Best LightGBM Parameters:", lgb_gs.best_params_)






After GridSearch 

In [ ]:
lgb_bestParams = {
    'num_leaves': 255,
    'learning_rate': 0.05,
    'n_estimators': 300
} 

#  Number of folds
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#  Storage for OOF predictions
oof_preds_lgb = np.zeros(X_train.shape[0]) 
test_preds_lgb = np.zeros(X_test.shape[0])

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold+1}/{n_folds}...")

    #  Split data
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    #  Train LightGBM model with best parameters
    model = lgb.LGBMClassifier(**lgb_bestParams)
    model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],  #  Validation set
    eval_metric="logloss",  #  Add evaluation metric
    callbacks=[lgb.early_stopping(10,verbose=True)]
    )

    #  Predict OOF for this fold
    oof_preds_lgb[valid_idx] = model.predict_proba(X_val)[:, 1]

    #  Predict test set (average across folds)
    test_preds_lgb += model.predict_proba(X_test)[:, 1] / n_folds

#  Compute final accuracy (on OOF predictions)
oof_accuracy = accuracy_score(y_train, (oof_preds_lgb > 0.5).astype(int))


#  Evaluate XGBoost Test Accuracy
test_preds_lgb_binary = (test_preds_lgb > 0.5).astype(int)
test_accuracy_lgb = accuracy_score(y_test, test_preds_lgb_binary)


print(f"OOF Accuracy: {oof_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy_lgb:.4f}") 

#* Now `oof_preds` and `test_preds` are ready for stacking

Training fold 1/5...
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2788518, number of negative: 4680145
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.468671 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 34158
[LightGBM] [Info] Number of data points in the train set: 7468663, number of used features: 44
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.373362 -> initscore=-0.517819
[LightGBM] [Info] Start training from score -0.517819
Training until validation scores don't improve for 10 rounds
Did not meet early stoppin

XGBOOST  
GRID SEARCH WITH XGBOOST (Not Performed In this script) - task separated out for another team, but  the following params are used based on the best params found by them on gridsearch

GET MY RESULTS 

NOW I TRAIN XGBOOST WITH K FOLD WITH BY RIGHT BEST PARAMS

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import gc 
import torch 
 
gc.collect()
torch.cuda.empty_cache() # clear my vram


xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'learning_rate': 0.2,
    'max_depth': 9,
    'n_estimators': 200,
    'gamma': 1.0, 
    'colsample_bytree': 1.0, 
    'subsample' : 1.0, 
    'enable_categorical': True ,
    'tree_method': 'gpu_hist',

}

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds_xgb = np.zeros(X_train.shape[0])  # OOF storage
test_preds_xgb = np.zeros(X_test.shape[0])  # Test storage

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold+1}/{n_folds}...")

    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    #  Train XGBoost model
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=True  
    )

    #  Store OOF predictions
    oof_preds_xgb[valid_idx] = model.predict_proba(X_val)[:, 1]

    #  Predict on test set and average across folds
    test_preds_xgb += model.predict_proba(X_test)[:, 1] / n_folds

#  Evaluate XGBoost OOF Accuracy
oof_accuracy_xgb = accuracy_score(y_train, (oof_preds_xgb > 0.5).astype(int))

#  Evaluate XGBoost Test Accuracy
test_preds_xgb_binary = (test_preds_xgb > 0.5).astype(int)
test_accuracy_xgb = accuracy_score(y_test, test_preds_xgb_binary)


print(f"XGBoost OOF Accuracy: {oof_accuracy_xgb:.4f}")
print(f"XGBoost Test Accuracy: {test_accuracy_xgb:.4f}")  # ✅ Now showing test accuracy


Training fold 1/5...


c:\Users\yugik\OneDrive - Singapore Institute Of Technology\Y2 T2\INF2008 MACHINE LEARNING\Project\ProjectTesting\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:57:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.64072
[1]	validation_0-logloss:0.62578
[2]	validation_0-logloss:0.61484
[3]	validation_0-logloss:0.60641
[4]	validation_0-logloss:0.60024
[5]	validation_0-logloss:0.59490
[6]	validation_0-logloss:0.59027
[7]	validation_0-logloss:0.58719
[8]	validation_0-logloss:0.58363
[9]	validation_0-logloss:0.58187
[10]	validation_0-logloss:0.57871
[11]	validation_0-logloss:0.57692
[12]	validation_0-logloss:0.57533
[13]	validation_0-logloss:0.57323
[14]	validation_0-logloss:0.57148
[15]	validation_0-logloss:0.56985
[16]	validation_0-logloss:0.56874
[17]	validation_0-logloss:0.56811
[18]	validation_0-logloss:0.56737
[19]	validation_0-logloss:0.56609
[20]	validation_0-logloss:0.56472
[21]	validation_0-logloss:0.56282
[22]	validation_0-logloss:0.56156
[23]	validation_0-logloss:0.56092
[24]	validation_0-logloss:0.55998
[25]	validation_0-logloss:0.55929
[26]	validation_0-logloss:0.55860
[27]	validation_0-logloss:0.55818
[28]	validation_0-logloss:0.55728
[29]	validation_0-loglos

c:\Users\yugik\OneDrive - Singapore Institute Of Technology\Y2 T2\INF2008 MACHINE LEARNING\Project\ProjectTesting\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:58:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\yugik\OneDrive - Singapore Institute Of Technology\Y2 T2\INF2008 MACHINE LEARNING\Project\ProjectTesting\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [02:58:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. 

Training fold 2/5...


...

Training fold 5/5...


c:\Users\yugik\OneDrive - Singapore Institute Of Technology\Y2 T2\INF2008 MACHINE LEARNING\Project\ProjectTesting\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [03:00:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.64072
[1]	validation_0-logloss:0.62577
[2]	validation_0-logloss:0.61473
[3]	validation_0-logloss:0.60615
[4]	validation_0-logloss:0.59960
[5]	validation_0-logloss:0.59509
[6]	validation_0-logloss:0.59041
[7]	validation_0-logloss:0.58732
[8]	validation_0-logloss:0.58438
[9]	validation_0-logloss:0.58160
[10]	validation_0-logloss:0.57896
[11]	validation_0-logloss:0.57742
[12]	validation_0-logloss:0.57536
[13]	validation_0-logloss:0.57362
[14]	validation_0-logloss:0.57186
[15]	validation_0-logloss:0.57074
[16]	validation_0-logloss:0.56860
[17]	validation_0-logloss:0.56692
[18]	validation_0-logloss:0.56636
[19]	validation_0-logloss:0.56487
[20]	validation_0-logloss:0.56375
[21]	validation_0-logloss:0.56273
[22]	validation_0-logloss:0.56213
[23]	validation_0-logloss:0.56130
[24]	validation_0-logloss:0.56023
[25]	validation_0-logloss:0.56002
[26]	validation_0-logloss:0.55910
[27]	validation_0-logloss:0.55877
[28]	validation_0-logloss:0.55838
[29]	validation_0-loglos

c:\Users\yugik\OneDrive - Singapore Institute Of Technology\Y2 T2\INF2008 MACHINE LEARNING\Project\ProjectTesting\env\Lib\site-packages\xgboost\core.py:158: UserWarning: [03:01:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBoost OOF Accuracy: 0.7298
XGBoost Test Accuracy: 0.7347


CATBOOST
  
GRID SEARCH WITH CATBOOST (Not Performed In this script) - task separated out for another team, but  the following params are used based on the best params found by them on gridsearch

In [15]:
X_train[cat_cols].columns.tolist()

['Airline Code',
 'Aircraft Registration',
 'Operator',
 'Type Code',
 'Mode S',
 'Serial Number',
 'FROM',
 'TO']

In [11]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'learning_rate': 0.2,
    'depth': 8,                   # Reduced depth for faster training
    'iterations': 500,            # Further reduced iterations
    'colsample_bylevel': 1.0,
    'random_strength': 0,
    'l2_leaf_reg': 3,
    'task_type': 'CPU',
    'verbose': 50,
    'allow_writing_files': False,
    'thread_count': 4,
    'early_stopping_rounds': 30   # Adjust early stopping rounds if needed
}

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds_cat = np.zeros(X_train.shape[0])
test_preds_cat = np.zeros(X_test.shape[0])

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train)):
    print(f"Training fold {fold+1}/{n_folds}...")

    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    # Create CatBoost model
    model_cat = CatBoostClassifier(**cat_params)
    model_cat.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        cat_features=X_train[cat_cols].columns.tolist(),
        use_best_model=True  # Early stopping if it doesn't improve
    )

    # OOF predictions
    oof_preds_cat[valid_idx] = model_cat.predict_proba(X_val)[:, 1]

    # Test predictions
    test_preds_cat += model_cat.predict_proba(X_test)[:, 1] / n_folds

# Evaluate CatBoost OOF Accuracy
oof_accuracy_cat = accuracy_score(y_train, (oof_preds_cat > 0.5).astype(int))

# Evaluate CatBoost Test Accuracy
test_preds_cat_binary = (test_preds_cat > 0.5).astype(int)
test_accuracy_cat = accuracy_score(y_test, test_preds_cat_binary)

print(f"CatBoost OOF Accuracy: {oof_accuracy_cat:.4f}")
print(f"CatBoost Test Accuracy: {test_accuracy_cat:.4f}")


Training fold 1/5...
0:	learn: 0.6539931	test: 0.6531735	best: 0.6531735 (0)	total: 6.51s	remaining: 54m 9s
50:	learn: 0.5415611	test: 0.5357743	best: 0.5357743 (50)	total: 5m 17s	remaining: 46m 31s
100:	learn: 0.5318352	test: 0.5260291	best: 0.5260291 (100)	total: 10m 41s	remaining: 42m 16s
150:	learn: 0.5269198	test: 0.5211274	best: 0.5211274 (150)	total: 16m 3s	remaining: 37m 5s
200:	learn: 0.5230662	test: 0.5172855	best: 0.5172855 (200)	total: 21m 23s	remaining: 31m 49s
250:	learn: 0.5202906	test: 0.5146718	best: 0.5146718 (250)	total: 26m 40s	remaining: 26m 28s
300:	learn: 0.5177140	test: 0.5121850	best: 0.5121850 (300)	total: 32m 14s	remaining: 21m 18s
350:	learn: 0.5155545	test: 0.5100991	best: 0.5100991 (350)	total: 38m 30s	remaining: 16m 20s
400:	learn: 0.5136194	test: 0.5082721	best: 0.5082721 (400)	total: 44m 28s	remaining: 10m 58s
450:	learn: 0.5121611	test: 0.5070455	best: 0.5070455 (450)	total: 50m 19s	remaining: 5m 28s
499:	learn: 0.5107110	test: 0.5057485	best: 0.505748

**STACK THIS UP**

LOGISTIC REGRESSION

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import numpy as np

# Combine OOF predictions from all base models
meta_X_train = np.column_stack((oof_preds_xgb, oof_preds_lgb, oof_preds_cat))
meta_y_train = y_train  # target remains the same

# Train Logistic Regression Meta-Model
meta_model = LogisticRegression()
meta_model.fit(meta_X_train, meta_y_train)

# Combine Test Predictions for Final Prediction
meta_X_test = np.column_stack((test_preds_xgb, test_preds_lgb, test_preds_cat))
final_test_preds = meta_model.predict(meta_X_test)  # Predict final test labels

# Evaluate Meta-Model Performance
final_accuracy = accuracy_score(y_test, final_test_preds)
cm = confusion_matrix(y_test, final_test_preds)
f1 = f1_score(y_test, final_test_preds)
precision = precision_score(y_test, final_test_preds)
recall = recall_score(y_test, final_test_preds)

print(f"Final Stacked Model Test Accuracy (Logistic Regression): {final_accuracy:.4f}")
print("Confusion Matrix:")
print(cm)
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Final Stacked Model Test Accuracy (Logistic Regression): 0.7516
Confusion Matrix:
[[1263176  199388]
 [ 380442  490952]]
F1 Score: 0.6287
Precision: 0.7112
Recall: 0.5634


SGD

In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import numpy as np

# Combine OOF predictions from all base models
meta_X_train = np.column_stack((oof_preds_xgb, oof_preds_lgb, oof_preds_cat))
meta_y_train = y_train  # target remains the same

# Train SGDClassifier as Meta-Model with logistic regression loss
meta_model = SGDClassifier(loss="log_loss", max_iter=1000, tol=1e-3, random_state=42)
meta_model.fit(meta_X_train, meta_y_train)

# Combine Test Predictions for Final Prediction
meta_X_test = np.column_stack((test_preds_xgb, test_preds_lgb, test_preds_cat))
final_test_preds = meta_model.predict(meta_X_test)  # Predict final test labels

# Evaluate Meta-Model Performance
final_accuracy = accuracy_score(y_test, final_test_preds)
cm = confusion_matrix(y_test, final_test_preds)
f1 = f1_score(y_test, final_test_preds)
precision = precision_score(y_test, final_test_preds)
recall = recall_score(y_test, final_test_preds)

print(f"Final Stacked Model Test Accuracy (SGDClassifier): {final_accuracy:.4f}")
print("Confusion Matrix:")
print(cm)
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Final Stacked Model Test Accuracy (SGDClassifier): 0.7514
Confusion Matrix:
[[1266665  195899]
 [ 384426  486968]]
F1 Score: 0.6266
Precision: 0.7131
Recall: 0.5588
